In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn import tree
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
date_columns = [204, 75, 73, 217, 158, 159, 156, 157, 176, 177, 178, 179, 166, 167, 168, 169]
def parse_date(x):
    return pd.to_datetime(x, format="%d%b%y:%H:%M:%S")  

raw_data = pd.read_csv('train.csv',                    
                   parse_dates=date_columns,
                   #nrows=50000,
                   date_parser = parse_date)

test_data = pd.read_csv('test.csv',                    
                   parse_dates=date_columns,
                   #nrows=50000,
                   date_parser = parse_date)



<class 'pandas.core.frame.DataFrame'>
Int64Index: 145231 entries, 0 to 145230
Columns: 1934 entries, ID to target
dtypes: datetime64[ns](16), float64(477), int64(1406), object(35)
memory usage: 2.1+ GB


/Library/Python/2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (8,9,10,11,12,43,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
raw_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145231 entries, 0 to 145230
Columns: 1934 entries, ID to target
dtypes: datetime64[ns](16), float64(477), int64(1406), object(35)
memory usage: 2.1+ GB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 145232 entries, 0 to 145231
Columns: 1933 entries, ID to VAR_1934
dtypes: datetime64[ns](16), float64(477), int64(1405), object(35)
memory usage: 2.1+ GB


In [10]:

def factorize(df):
    columns_by_dtype = {}
    for dtype in raw_data.dtypes.unique(): 
        columns_by_dtype[dtype] = df[df.dtypes[df.dtypes == dtype].index]
    
    object_df = pd.concat([ pd.DataFrame({c: df[c].factorize()[0]}) for c in columns_by_dtype[np.dtype('O')].columns ], axis=1)    
    #boolean_df = pd.concat([ pd.DataFrame({c: df[c].factorize()[0]}) for c in columns_by_dtype[np.dtype('bool')].columns ], axis=1)    
    normalized_df = pd.concat([
            object_df, 
            columns_by_dtype[np.dtype('float64')],
            columns_by_dtype[np.dtype('int64')]
        ], axis=1)
    
    
    return normalized_df

In [17]:
all_data = pd.concat([raw_data, test_data], ignore_index=True)

In [18]:
all_data = factorize(all_data)

In [46]:
training_data = all_data[all_data['target'].notnull()]
test_data = all_data[all_data['target'].isnull()]

In [38]:
t1_features = training_data.drop('target', axis=1).drop('ID', axis=1).fillna(-1)
t1_targets = training_data['target']

In [39]:
n = 130000
clf = tree.DecisionTreeClassifier().fit(t1_features[:n], t1_targets[:n])
clf.score(t1_features[n:], t1_targets[n:])


0.69844396297025801

In [59]:
test_features = test_data.drop('target', axis=1).drop('ID', axis=1).fillna(-1)
test_targets =  pd.Series(clf.predict(test_features)) 

In [72]:
submission = pd.concat([test_data['ID'].reset_index(), test_targets], ignore_index=True, axis=1)[[1,2]]
submission.columns = ['ID', 'target']
submission.astype(np.int).to_csv('submission.csv', index=False)

In [74]:
submission['ID']

0              1
1              3
2              6
3              9
4             10
5             11
6             12
7             13
8             15
9             17
10            18
11            19
12            27
13            29
14            33
15            34
16            39
17            41
18            44
19            45
20            47
21            48
22            49
23            53
24            56
25            57
26            59
27            61
28            64
29            68
           ...  
145202    290400
145203    290401
145204    290403
145205    290406
145206    290408
145207    290410
145208    290411
145209    290413
145210    290416
145211    290418
145212    290419
145213    290420
145214    290421
145215    290422
145216    290423
145217    290425
145218    290428
145219    290430
145220    290433
145221    290435
145222    290437
145223    290438
145224    290442
145225    290444
145226    290446
145227    290451
145228    290455
145229    2904